In [1]:
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, Runnable
from langchain_core.output_parsers import StrOutputParser
from embedding.zhipu_embedding import ZhipuAIEmbeddings
from langchain.vectorstores.chroma import Chroma
from utils.path_utils import get_vector_store_dir
# from retriever import retriever
from utils.env_loader import get_env
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [2]:
api_key = get_env("ZHIPUAI_API_KEY")
model = ChatOpenAI(
    temperature=0.95,
    model="glm-4",
    openai_api_key=api_key,
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)

vector_db = Chroma(
    persist_directory=get_vector_store_dir(),
    embedding_function=ZhipuAIEmbeddings(),
)

template = """
使用下文内容来回答问题，不知道则直接回答不知道，不要试图编造答案。最多使用三句话，答案简明扼要。总在回答的最后说"谢谢你的提问！"。
{context}
问题:{question}
"""

QA_CHAIN_PROMPT=PromptTemplate(input_variables=["context", "question"], template=template)

# 加载检索器
qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vector_db.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

/tmp/ipykernel_46985/1611650476.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  model = ChatOpenAI(
/tmp/ipykernel_46985/1611650476.py:9: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vector_db = Chroma(


In [3]:
print(vector_db._collection.count())

869


In [4]:
result = qa_chain({"query":"什么是大模型"})

/tmp/ipykernel_46985/2071198752.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa_chain({"query":"什么是大模型"})


In [5]:
print(result["result"])

大模型，通常指的是参数规模极大的机器学习模型，例如大型语言模型（LLM），它们能在大量数据上学习复杂的模式，用于预测下一个单词或执行特定任务。这些模型具有强大的泛化和应用能力，可以应用于聊天机器人、文本转换、推理等多个领域。谢谢你的提问！
